# LangSmith 向量检索评估教程

## 目录
1. [LangSmith平台概述](#概述)
2. [安装和环境配置](#安装)
3. [LangSmith评估器类型](#评估器类型)
4. [创建数据集](#创建数据集)
5. [向量检索评估器实现](#评估器实现)
6. [运行评估](#运行评估)
7. [完整示例代码](#完整示例)
8. [结果分析和优化建议](#分析优化)


## 1. LangSmith平台概述 {#概述}

LangSmith是LangChain官方提供的评估、调试和监控平台，专为LLM应用设计。

### 核心特点：
- ✅ **可视化界面**：提供直观的评估结果展示和分析
- ✅ **多种评估器**：支持LLM-as-Judge、自定义评估器、嵌入距离等
- ✅ **数据集管理**：方便管理和版本控制评估数据集
- ✅ **实验追踪**：记录每次评估的结果，便于对比和优化
- ✅ **生产监控**：可以监控生产环境的RAG系统性能

### LangSmith评估流程：
```
创建数据集 → 定义评估器 → 运行评估
              ↓
        LangSmith UI查看结果
              ↓
        分析并优化RAG系统
```


## 2. 安装和环境配置 {#安装}


In [ ]:
# 安装LangSmith和相关依赖
%pip install langsmith langchain-openai langchain-community -q


In [ ]:
# 配置环境变量
import os
from dotenv import load_dotenv

# 加载.env文件（如果存在）
load_dotenv()

# 设置LangSmith API密钥（需要从 https://smith.langchain.com/ 获取）
# os.environ["LANGSMITH_API_KEY"] = "your-langsmith-api-key"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "rag-evaluation"  # 项目名称

# 设置OpenAI API密钥（用于LLM评估器）
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# 导入必要的库
from langsmith import Client
from langchain_openai import ChatOpenAI
from typing_extensions import Annotated, TypedDict
from typing import Dict, List, Any
import pandas as pd
import numpy as np

print("环境配置完成！")
print("\n注意：请确保已设置以下环境变量：")
print("  - LANGSMITH_API_KEY")
print("  - OPENAI_API_KEY (如果使用OpenAI作为评估器)")


## 3. LangSmith评估器类型 {#评估器类型}

LangSmith支持多种类型的评估器：

### 3.1 LLM-as-Judge评估器
使用大语言模型作为评判者，自动评估答案质量。这是最常用的评估方式。

**优点**：
- 自动化程度高
- 可以评估复杂的主观性问题
- 不需要人工标注

**缺点**：
- 需要调用LLM API，有成本
- 评估结果可能有一定随机性

### 3.2 自定义评估器
根据具体需求编写评估逻辑，可以结合规则、语义相似度等多种方法。

**适用场景**：
- 有明确的评估规则
- 需要评估特定维度
- 希望控制评估成本

### 3.3 嵌入距离评估器
使用嵌入向量计算相似度，评估检索结果的相关性。

**适用场景**：
- 评估检索质量
- 快速评估大量样本
- 不需要深度语义理解时


## 4. 创建数据集 {#创建数据集}

在LangSmith中，我们需要先创建数据集，然后对数据集进行评估。


In [ ]:
# 初始化LangSmith客户端
# 注意：需要先设置LANGSMITH_API_KEY环境变量
try:
    client = Client()
    print("LangSmith客户端初始化成功！")
except Exception as e:
    print(f"初始化失败：{str(e)}")
    print("请确保已设置LANGSMITH_API_KEY环境变量")
    print("可以在 https://smith.langchain.com/ 获取API密钥")


In [ ]:
# 创建评估数据集
def create_evaluation_dataset(client, dataset_name: str, examples: List[Dict]):
    """
    创建LangSmith评估数据集
    
    Args:
        client: LangSmith客户端
        dataset_name: 数据集名称
        examples: 示例列表，每个示例包含inputs和outputs
    
    Returns:
        str: 数据集名称
    """
    try:
        # 检查数据集是否已存在
        try:
            dataset = client.read_dataset(dataset_name=dataset_name)
            print(f"数据集 '{dataset_name}' 已存在，将添加新示例")
            dataset_id = dataset.id
        except:
            # 创建新数据集
            dataset = client.create_dataset(
                dataset_name=dataset_name,
                description="RAG系统评估数据集"
            )
            dataset_id = dataset.id
            print(f"创建新数据集 '{dataset_name}'")
        
        # 添加示例
        for example in examples:
            client.create_example(
                inputs=example["inputs"],
                outputs=example.get("outputs", {}),
                dataset_id=dataset_id
            )
        
        print(f"已添加 {len(examples)} 个示例到数据集")
        return dataset_name
    
    except Exception as e:
        print(f"创建数据集时出错：{str(e)}")
        return None

# 准备示例数据
evaluation_examples = [
    {
        "inputs": {
            "question": "RAG是什么？",
            "documents": [
                "RAG（Retrieval-Augmented Generation）是一种结合检索和生成的技术。",
                "RAG的核心思想是在生成答案前，先从知识库中检索相关文档。",
                "RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识。"
            ]
        },
        "outputs": {
            "answer": "RAG是一种结合检索和生成的技术，它通过在生成答案前从知识库检索相关文档来解决大模型的知识盲区问题。"
        }
    },
    {
        "inputs": {
            "question": "RAG的核心思想是什么？",
            "documents": [
                "RAG的核心思想是在生成答案前，先从知识库中检索相关文档。",
                "检索到的文档作为上下文输入给生成模型，帮助生成更准确的答案。"
            ]
        },
        "outputs": {
            "answer": "RAG的核心思想是在生成答案前先从知识库检索相关文档，然后将检索到的文档作为上下文输入给生成模型。"
        }
    },
    {
        "inputs": {
            "question": "RAG能解决什么问题？",
            "documents": [
                "RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识。",
                "通过RAG，模型可以动态检索最新的信息，而不需要重新训练。"
            ]
        },
        "outputs": {
            "answer": "RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识，并且可以动态检索最新信息而不需要重新训练。"
        }
    }
]

# 创建数据集（如果已初始化客户端）
if 'client' in locals():
    dataset_name = create_evaluation_dataset(
        client=client,
        dataset_name="rag-evaluation-dataset",
        examples=evaluation_examples
    )
else:
    print("跳过数据集创建（客户端未初始化）")
    dataset_name = "rag-evaluation-dataset"  # 示例名称


## 5. 向量检索评估器实现 {#评估器实现}

本节实现多个专门用于评估向量检索的评估器。


In [ ]:
# 定义评估结果的类型
class CorrectnessGrade(TypedDict):
    """答案正确性评估结果"""
    explanation: Annotated[str, "评分理由"]
    correct: Annotated[bool, "答案是否正确"]

class GroundednessGrade(TypedDict):
    """答案基于文档的评估结果"""
    explanation: Annotated[str, "评分理由"]
    grounded: Annotated[bool, "答案是否基于文档"]

class RelevanceGrade(TypedDict):
    """答案相关性评估结果"""
    explanation: Annotated[str, "评分理由"]
    relevant: Annotated[bool, "答案是否相关"]

class RetrievalPrecisionGrade(TypedDict):
    """检索精确度评估结果"""
    explanation: Annotated[str, "评分理由"]
    precision_score: Annotated[float, "精确度分数 (0-1)"]

class RetrievalRecallGrade(TypedDict):
    """检索召回率评估结果"""
    explanation: Annotated[str, "评分理由"]
    recall_score: Annotated[float, "召回率分数 (0-1)"]

# 初始化LLM评估器（使用结构化输出）
try:
    grader_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
        CorrectnessGrade, method="json_schema", strict=True
    )
    print("LLM评估器初始化成功！")
except Exception as e:
    print(f"LLM评估器初始化失败：{str(e)}")
    print("请确保已设置OPENAI_API_KEY环境变量")
    grader_llm = None


In [ ]:
# 1. Correctness Evaluator（答案正确性评估器）
def correctness_evaluator(inputs: dict, outputs: dict, reference_outputs: dict) -> dict:
    """
    对比答案的事实准确性
    
    Args:
        inputs: 包含question和documents的字典
        outputs: 包含answer的字典
        reference_outputs: 包含标准答案的字典
    
    Returns:
        dict: 包含correctness评分的字典
    """
    if grader_llm is None:
        return {"correctness": False, "reason": "LLM评估器未初始化"}
    
    prompt = f"""
    问题：{inputs['question']}
    参考答案：{reference_outputs.get('answer', '')}
    生成答案：{outputs['answer']}
    
    请评估生成答案的事实准确性。生成答案是否与参考答案在事实层面一致？
    
    注意：
    - 如果生成答案和参考答案在核心事实上一致，即使表达方式不同，也应判定为正确
    - 如果生成答案包含错误信息或遗漏关键信息，应判定为不正确
    """
    
    try:
        grade = grader_llm.invoke([{"role": "user", "content": prompt}])
        return {
            "correctness": grade.get("correct", False),
            "explanation": grade.get("explanation", "")
        }
    except Exception as e:
        return {"correctness": False, "reason": f"评估失败: {str(e)}"}

print("Correctness Evaluator已定义")


In [ ]:
# 2. Groundedness Evaluator（答案基于文档的评估器）
def groundedness_evaluator(inputs: dict, outputs: dict) -> dict:
    """
    检测答案是否基于检索文档，没有幻觉
    
    Args:
        inputs: 包含question和documents的字典
        outputs: 包含answer的字典
    
    Returns:
        dict: 包含groundedness评分的字典
    """
    if grader_llm is None:
        return {"groundedness": False, "reason": "LLM评估器未初始化"}
    
    # 将文档列表转换为字符串
    doc_string = "\n\n".join([
        f"文档 {i+1}: {doc}" 
        for i, doc in enumerate(inputs.get('documents', []))
    ])
    
    prompt = f"""
    文档：
    {doc_string}
    
    答案：{outputs['answer']}
    
    请评估答案是否完全基于上述文档，没有引入文档中没有的信息（幻觉）？
    
    注意：
    - 如果答案中的所有信息都可以从文档中推断出来，判定为基于文档
    - 如果答案包含文档中没有的信息，判定为包含幻觉
    - 合理的推理和总结是允许的，但不能添加新的事实
    """
    
    try:
        # 使用临时的结构化输出
        groundedness_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
            GroundednessGrade, method="json_schema", strict=True
        )
        grade = groundedness_llm.invoke([{"role": "user", "content": prompt}])
        return {
            "groundedness": grade.get("grounded", False),
            "explanation": grade.get("explanation", "")
        }
    except Exception as e:
        return {"groundedness": False, "reason": f"评估失败: {str(e)}"}

print("Groundedness Evaluator已定义")


In [ ]:
# 3. Relevance Evaluator（答案相关性评估器）
def relevance_evaluator(inputs: dict, outputs: dict) -> dict:
    """
    检测答案与问题的相关性
    
    Args:
        inputs: 包含question的字典
        outputs: 包含answer的字典
    
    Returns:
        dict: 包含relevance评分的字典
    """
    if grader_llm is None:
        return {"relevance": False, "reason": "LLM评估器未初始化"}
    
    prompt = f"""
    问题：{inputs['question']}
    答案：{outputs['answer']}
    
    请评估答案是否有效回答了问题？
    
    注意：
    - 如果答案直接回答了问题，即使不够完整，也应判定为相关
    - 如果答案答非所问或偏离主题，应判定为不相关
    - 考虑答案的完整性和相关性
    """
    
    try:
        relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
            RelevanceGrade, method="json_schema", strict=True
        )
        grade = relevance_llm.invoke([{"role": "user", "content": prompt}])
        return {
            "relevance": grade.get("relevant", False),
            "explanation": grade.get("explanation", "")
        }
    except Exception as e:
        return {"relevance": False, "reason": f"评估失败: {str(e)}"}

print("Relevance Evaluator已定义")


In [ ]:
# 4. Retrieval Precision Evaluator（检索精确度评估器）
def retrieval_precision_evaluator(inputs: dict, outputs: dict) -> dict:
    """
    评估检索到的文档的相关性（精确度）
    
    Args:
        inputs: 包含question和documents的字典
        outputs: 包含answer的字典（用于判断文档相关性）
    
    Returns:
        dict: 包含precision_score的字典
    """
    if grader_llm is None:
        return {"precision_score": 0.0, "reason": "LLM评估器未初始化"}
    
    documents = inputs.get('documents', [])
    if not documents:
        return {"precision_score": 0.0, "reason": "没有检索到文档"}
    
    # 评估每个文档的相关性
    relevant_count = 0
    total_count = len(documents)
    
    doc_string = "\n\n".join([
        f"文档 {i+1}: {doc}" 
        for i, doc in enumerate(documents)
    ])
    
    prompt = f"""
    问题：{inputs['question']}
    答案：{outputs['answer']}
    
    检索到的文档：
    {doc_string}
    
    请评估每个文档对回答这个问题的相关性。对于每个文档，判断它是否有助于回答这个问题。
    
    请返回一个分数，表示相关文档的比例（0-1之间）。
    例如，如果5个文档中有3个相关，则返回0.6。
    """
    
    try:
        precision_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
            RetrievalPrecisionGrade, method="json_schema", strict=True
        )
        grade = precision_llm.invoke([{"role": "user", "content": prompt}])
        return {
            "precision_score": grade.get("precision_score", 0.0),
            "explanation": grade.get("explanation", "")
        }
    except Exception as e:
        # 如果评估失败，使用简单的启发式方法
        return {"precision_score": 0.5, "reason": f"评估失败: {str(e)}"}

print("Retrieval Precision Evaluator已定义")


In [ ]:
# 5. Retrieval Recall Evaluator（检索召回率评估器）
def retrieval_recall_evaluator(inputs: dict, outputs: dict, reference_outputs: dict) -> dict:
    """
    评估检索到的文档是否覆盖了所有相关信息（召回率）
    
    Args:
        inputs: 包含question和documents的字典
        outputs: 包含answer的字典
        reference_outputs: 包含ground_truth_documents的字典
    
    Returns:
        dict: 包含recall_score的字典
    """
    if grader_llm is None:
        return {"recall_score": 0.0, "reason": "LLM评估器未初始化"}
    
    retrieved_docs = inputs.get('documents', [])
    ground_truth_docs = reference_outputs.get('ground_truth_documents', [])
    
    if not ground_truth_docs:
        return {"recall_score": 0.0, "reason": "没有提供ground truth文档"}
    
    retrieved_string = "\n\n".join([
        f"检索文档 {i+1}: {doc}" 
        for i, doc in enumerate(retrieved_docs)
    ])
    
    ground_truth_string = "\n\n".join([
        f"参考文档 {i+1}: {doc}" 
        for i, doc in enumerate(ground_truth_docs)
    ])
    
    prompt = f"""
    问题：{inputs['question']}
    
    检索到的文档：
    {retrieved_string}
    
    应该检索到的参考文档（ground truth）：
    {ground_truth_string}
    
    请评估检索到的文档是否覆盖了所有应该检索到的参考文档中的信息。
    
    返回一个分数（0-1之间），表示检索到的文档覆盖了多少参考文档中的信息。
    例如，如果参考文档中有3个关键信息点，检索到的文档覆盖了2个，则返回0.67。
    """
    
    try:
        recall_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
            RetrievalRecallGrade, method="json_schema", strict=True
        )
        grade = recall_llm.invoke([{"role": "user", "content": prompt}])
        return {
            "recall_score": grade.get("recall_score", 0.0),
            "explanation": grade.get("explanation", "")
        }
    except Exception as e:
        return {"recall_score": 0.0, "reason": f"评估失败: {str(e)}"}

print("Retrieval Recall Evaluator已定义")


## 6. 运行评估 {#运行评估}

在LangSmith中，我们可以使用`client.evaluate()`方法来运行评估。该方法会：
1. 对数据集中的每个示例运行RAG系统
2. 使用定义的评估器评估结果
3. 在LangSmith UI中展示评估结果


In [ ]:
# 创建一个简单的RAG系统函数用于评估
def simple_rag_system(question: str, documents: list = None) -> dict:
    """
    简单的RAG系统示例
    在实际使用中，应该替换为你的实际RAG系统
    
    Args:
        question: 用户问题
        documents: 检索到的文档列表
    
    Returns:
        dict: 包含answer的字典
    """
    # 如果没有提供文档，使用默认文档
    if documents is None:
        documents = [
            "RAG（Retrieval-Augmented Generation）是一种结合检索和生成的技术。",
            "RAG的核心思想是在生成答案前，先从知识库中检索相关文档。",
            "RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识。"
        ]
    
    # 这里应该是实际的LLM调用
    # 为了演示，返回一个示例答案
    answer = "RAG是一种结合检索和生成的技术，它通过在生成答案前从知识库检索相关文档来解决大模型的知识盲区问题。"
    
    return {"answer": answer}

print("RAG系统函数已创建")


In [ ]:
# 准备评估器列表
evaluators = [
    correctness_evaluator,
    groundedness_evaluator,
    relevance_evaluator,
    retrieval_precision_evaluator,
    # retrieval_recall_evaluator,  # 需要reference_outputs，可能需要特殊处理
]

print("已准备评估器列表")
print(f"评估器数量：{len(evaluators)}")


In [ ]:
# 运行评估（如果客户端已初始化）
if 'client' in locals() and dataset_name:
    try:
        print(f"开始评估数据集：{dataset_name}")
        print("注意：这将调用LLM API，可能需要一些时间...")
        
        # 运行评估
        experiment_results = client.evaluate(
            target=simple_rag_system,  # 目标函数
            data=dataset_name,  # 数据集名称
            evaluators=evaluators,  # 评估器列表
            experiment_prefix="rag-evaluation",  # 实验前缀
            max_concurrency=2,  # 最大并发数
        )
        
        print("\n评估完成！")
        print(f"评估结果数量：{len(list(experiment_results))}")
        print("\n你可以在LangSmith UI中查看详细结果：")
        print("https://smith.langchain.com/")
        
    except Exception as e:
        print(f"运行评估时出错：{str(e)}")
        print("\n可能的原因：")
        print("1. 未设置LANGSMITH_API_KEY环境变量")
        print("2. 数据集不存在或格式不正确")
        print("3. 评估器函数签名不匹配")
        print("4. 网络连接问题")
else:
    print("跳过评估（客户端未初始化或数据集不存在）")
    print("\n示例代码：")
    print("""
    # 运行评估的代码示例：
    experiment_results = client.evaluate(
        target=your_rag_system_function,
        data="your-dataset-name",
        evaluators=[correctness_evaluator, groundedness_evaluator, ...],
        experiment_prefix="rag-evaluation",
        max_concurrency=2,
    )
    """)


In [ ]:
# 本地评估示例（不依赖LangSmith服务器）
# 这个方法可以在没有LangSmith API的情况下进行本地评估

def local_evaluate(rag_system, test_dataset, evaluators):
    """
    本地评估函数，不依赖LangSmith服务器
    
    Args:
        rag_system: RAG系统函数
        test_dataset: 测试数据集
        evaluators: 评估器列表
    
    Returns:
        list: 评估结果列表
    """
    results = []
    
    for idx, test_case in enumerate(test_dataset):
        print(f"\n处理测试用例 {idx + 1}/{len(test_dataset)}: {test_case['inputs']['question']}")
        
        # 运行RAG系统
        rag_output = rag_system(
            question=test_case['inputs']['question'],
            documents=test_case['inputs'].get('documents', [])
        )
        
        # 准备评估输入
        inputs = test_case['inputs']
        outputs = rag_output
        reference_outputs = test_case.get('outputs', {})
        
        # 运行每个评估器
        case_result = {
            'question': inputs['question'],
            'answer': outputs.get('answer', ''),
            'evaluations': {}
        }
        
        for evaluator in evaluators:
            try:
                # 根据评估器签名决定如何调用
                if 'reference_outputs' in evaluator.__code__.co_varnames:
                    eval_result = evaluator(inputs, outputs, reference_outputs)
                else:
                    eval_result = evaluator(inputs, outputs)
                
                # 提取评估器名称
                eval_name = evaluator.__name__.replace('_evaluator', '')
                case_result['evaluations'][eval_name] = eval_result
                
            except Exception as e:
                print(f"  评估器 {evaluator.__name__} 失败: {str(e)}")
        
        results.append(case_result)
    
    return results

# 如果有测试数据，可以进行本地评估
if 'evaluation_examples' in locals():
    print("开始本地评估...")
    local_results = local_evaluate(
        rag_system=simple_rag_system,
        test_dataset=evaluation_examples,
        evaluators=evaluators[:3]  # 只使用前3个评估器
    )
    
    print("\n" + "=" * 60)
    print("本地评估结果：")
    print("=" * 60)
    for result in local_results:
        print(f"\n问题：{result['question']}")
        print(f"答案：{result['answer'][:100]}...")
        print("评估结果：")
        for eval_name, eval_result in result['evaluations'].items():
            print(f"  {eval_name}: {eval_result}")
else:
    print("本地评估示例代码已准备就绪")


## 7. 完整示例代码 {#完整示例}

以下是一个完整的LangSmith RAG评估示例，展示了如何评估一个完整的RAG系统。


In [ ]:
# 完整的LangSmith RAG评估器类
class LangSmithRAGEvaluator:
    """完整的LangSmith RAG评估器"""
    
    def __init__(self, rag_system, client=None):
        """
        初始化评估器
        
        Args:
            rag_system: RAG系统函数或对象
            client: LangSmith客户端（可选）
        """
        self.rag_system = rag_system
        self.client = client
        
        # 定义评估器
        self.evaluators = {
            "correctness": correctness_evaluator,
            "groundedness": groundedness_evaluator,
            "relevance": relevance_evaluator,
            "retrieval_precision": retrieval_precision_evaluator,
            "retrieval_recall": retrieval_recall_evaluator,
        }
    
    def create_dataset(self, dataset_name: str, examples: List[Dict]):
        """
        创建LangSmith数据集
        
        Args:
            dataset_name: 数据集名称
            examples: 示例列表
        
        Returns:
            str: 数据集名称
        """
        if self.client is None:
            print("警告：LangSmith客户端未初始化，无法创建数据集")
            return None
        
        return create_evaluation_dataset(self.client, dataset_name, examples)
    
    def evaluate_on_langsmith(self, dataset_name: str, experiment_prefix: str = "rag-eval"):
        """
        在LangSmith平台上运行评估
        
        Args:
            dataset_name: 数据集名称
            experiment_prefix: 实验前缀
        
        Returns:
            Generator: 评估结果生成器
        """
        if self.client is None:
            raise ValueError("LangSmith客户端未初始化")
        
        return self.client.evaluate(
            target=self.rag_system,
            data=dataset_name,
            evaluators=list(self.evaluators.values()),
            experiment_prefix=experiment_prefix,
            max_concurrency=2,
        )
    
    def evaluate_locally(self, test_dataset: List[Dict]):
        """
        本地评估（不依赖LangSmith服务器）
        
        Args:
            test_dataset: 测试数据集
        
        Returns:
            DataFrame: 评估结果DataFrame
        """
        results = []
        
        for test_case in test_dataset:
            # 运行RAG系统
            inputs = test_case['inputs']
            rag_output = self.rag_system(
                question=inputs['question'],
                documents=inputs.get('documents', [])
            )
            
            outputs = rag_output if isinstance(rag_output, dict) else {"answer": str(rag_output)}
            reference_outputs = test_case.get('outputs', {})
            
            # 运行评估器
            case_result = {
                'question': inputs['question'],
                'answer': outputs.get('answer', ''),
            }
            
            for eval_name, evaluator in self.evaluators.items():
                try:
                    # 根据评估器签名决定如何调用
                    if 'reference_outputs' in evaluator.__code__.co_varnames:
                        eval_result = evaluator(inputs, outputs, reference_outputs)
                    else:
                        eval_result = evaluator(inputs, outputs)
                    
                    # 提取分数
                    if isinstance(eval_result, dict):
                        # 尝试提取分数
                        score_key = eval_name.replace('_', '_') + '_score'
                        if score_key in eval_result:
                            case_result[eval_name] = eval_result[score_key]
                        elif eval_name in eval_result:
                            case_result[eval_name] = eval_result[eval_name]
                        else:
                            # 尝试提取布尔值
                            for key in ['correct', 'grounded', 'relevant']:
                                if key in eval_result:
                                    case_result[eval_name] = eval_result[key]
                                    break
                    
                except Exception as e:
                    print(f"评估器 {eval_name} 失败: {str(e)}")
                    case_result[eval_name] = None
            
            results.append(case_result)
        
        return pd.DataFrame(results)

# 使用示例
print("LangSmith RAG评估器已创建")
print("\n使用方法：")
print("1. 初始化：evaluator = LangSmithRAGEvaluator(your_rag_system, client)")
print("2. 创建数据集：evaluator.create_dataset('my-dataset', examples)")
print("3. LangSmith评估：results = evaluator.evaluate_on_langsmith('my-dataset')")
print("4. 本地评估：results_df = evaluator.evaluate_locally(test_dataset)")


In [ ]:
# 示例：使用完整的评估器
if 'evaluation_examples' in locals():
    # 创建评估器实例
    evaluator = LangSmithRAGEvaluator(
        rag_system=simple_rag_system,
        client=client if 'client' in locals() else None
    )
    
    # 本地评估
    print("执行本地评估...")
    results_df = evaluator.evaluate_locally(evaluation_examples)
    
    print("\n" + "=" * 80)
    print("评估结果：")
    print("=" * 80)
    print(results_df.to_string(index=False))
    
    # 计算平均分数
    print("\n" + "=" * 80)
    print("平均评估分数：")
    print("=" * 80)
    numeric_columns = results_df.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        if col not in ['question']:
            avg_score = results_df[col].mean()
            print(f"{col:25s}: {avg_score:.4f}")
else:
    print("完整评估器已准备就绪，等待测试数据")


## 8. 结果分析和优化建议 {#分析优化}

### 8.1 评估结果解读

根据LangSmith评估结果，我们可以从以下几个方面进行分析：

1. **Correctness < 0.7**: 答案正确性不足
   - 优化建议：改进prompt，增加事实检查步骤；使用更强的LLM模型

2. **Groundedness < 0.7**: 答案包含幻觉或未基于文档
   - 优化建议：改进prompt，明确要求基于文档回答；增加文档验证步骤

3. **Relevance < 0.7**: 答案与问题相关性不足
   - 优化建议：改进问题理解；优化prompt中的回答要求

4. **Retrieval Precision < 0.7**: 检索到的文档相关性不高
   - 优化建议：改进embedding模型；增加rerank步骤；优化检索参数

5. **Retrieval Recall < 0.7**: 遗漏了相关文档
   - 优化建议：增加检索数量k；使用混合检索；改进检索策略


In [ ]:
# 评估结果分析函数
def analyze_langsmith_results(results_df):
    """
    分析LangSmith评估结果并给出优化建议
    
    Args:
        results_df: 包含评估结果的DataFrame
    
    Returns:
        dict: 分析结果和建议
    """
    analysis = {
        "summary": {},
        "recommendations": []
    }
    
    thresholds = {
        "correctness": 0.7,
        "groundedness": 0.7,
        "relevance": 0.7,
        "retrieval_precision": 0.7,
        "retrieval_recall": 0.7
    }
    
    for metric, threshold in thresholds.items():
        if metric in results_df.columns:
            # 处理布尔值列（转换为0/1）
            scores = results_df[metric]
            if scores.dtype == bool:
                scores = scores.astype(int)
            
            mean_score = scores.mean()
            analysis["summary"][metric] = {
                "mean": mean_score,
                "threshold": threshold,
                "status": "良好" if mean_score >= threshold else "需要改进"
            }
            
            if mean_score < threshold:
                if metric == "correctness":
                    analysis["recommendations"].append(
                        "Correctness得分较低：改进prompt，增加事实检查步骤，使用更强的LLM模型"
                    )
                elif metric == "groundedness":
                    analysis["recommendations"].append(
                        "Groundedness得分较低：改进prompt，明确要求基于文档回答，避免幻觉"
                    )
                elif metric == "relevance":
                    analysis["recommendations"].append(
                        "Relevance得分较低：改进问题理解，优化prompt中的回答要求"
                    )
                elif metric == "retrieval_precision":
                    analysis["recommendations"].append(
                        "Retrieval Precision得分较低：改进embedding模型，增加rerank步骤"
                    )
                elif metric == "retrieval_recall":
                    analysis["recommendations"].append(
                        "Retrieval Recall得分较低：增加检索数量k，使用混合检索策略"
                    )
    
    return analysis

# 如果有评估结果，进行分析
if 'results_df' in locals() and not results_df.empty:
    analysis = analyze_langsmith_results(results_df)
    
    print("=" * 60)
    print("评估结果分析：")
    print("=" * 60)
    
    print("\n指标概览：")
    for metric, info in analysis["summary"].items():
        status_icon = "✅" if info["status"] == "良好" else "⚠️"
        print(f"{status_icon} {metric:25s}: {info['mean']:.4f} (阈值: {info['threshold']:.2f}) - {info['status']}")
    
    if analysis["recommendations"]:
        print("\n优化建议：")
        for i, rec in enumerate(analysis["recommendations"], 1):
            print(f"{i}. {rec}")
    else:
        print("\n✅ 所有指标都达到阈值，系统表现良好！")
else:
    print("评估结果分析函数已准备就绪")


In [ ]:
# 可视化评估结果
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']  # 支持中文
matplotlib.rcParams['axes.unicode_minus'] = False

# 如果有评估结果，进行可视化
if 'results_df' in locals() and not results_df.empty:
    # 提取数值列
    numeric_cols = results_df.select_dtypes(include=[np.number]).columns.tolist()
    # 排除非评估指标列
    eval_cols = [col for col in numeric_cols if col not in ['question']]
    
    if eval_cols:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # 1. 各指标平均分数柱状图
        metric_means = [results_df[col].mean() for col in eval_cols]
        metric_names = [col.replace('_', ' ').title() for col in eval_cols]
        
        axes[0].bar(metric_names, metric_means, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'])
        axes[0].set_title('LangSmith评估指标平均分数', fontsize=14, fontweight='bold')
        axes[0].set_ylabel('分数', fontsize=12)
        axes[0].set_ylim([0, 1])
        axes[0].grid(axis='y', alpha=0.3)
        axes[0].tick_params(axis='x', rotation=45)
        
        # 添加数值标签
        for i, v in enumerate(metric_means):
            axes[0].text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontsize=10)
        
        # 2. 每个测试用例的分数趋势
        x = range(len(results_df))
        for col in eval_cols:
            scores = results_df[col]
            if scores.dtype == bool:
                scores = scores.astype(int)
            axes[1].plot(x, scores, marker='o', label=col.replace('_', ' ').title(), linewidth=2)
        
        axes[1].set_title('各测试用例评估分数趋势', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('测试用例编号', fontsize=12)
        axes[1].set_ylabel('分数', fontsize=12)
        axes[1].set_xticks(x)
        axes[1].set_xticklabels([f'Case {i+1}' for i in x])
        axes[1].legend(loc='best')
        axes[1].grid(alpha=0.3)
        axes[1].set_ylim([0, 1])
        
        plt.tight_layout()
        plt.show()
    else:
        print("没有可可视化的数值评估指标")
else:
    print("可视化代码已准备就绪，等待评估结果")


### 8.2 LangSmith UI使用指南

LangSmith提供了强大的Web界面来查看和分析评估结果：

1. **访问LangSmith UI**：
   - 网址：https://smith.langchain.com/
   - 使用你的API密钥登录

2. **查看评估结果**：
   - 在"Experiments"页面查看所有评估实验
   - 点击实验名称查看详细信息
   - 可以查看每个测试用例的评估分数

3. **分析功能**：
   - 对比不同实验的结果
   - 查看评估分数的分布
   - 导出评估报告

4. **持续监控**：
   - 设置定期评估任务
   - 监控生产环境的RAG系统性能
   - 设置告警阈值

### 8.3 最佳实践总结

1. **评估数据集构建**：
   - 包含不同类型的问题（简单事实、复杂推理、多跳问题）
   - 确保有足够的ground truth用于评估
   - 定期更新评估数据集

2. **评估频率**：
   - 每次模型或prompt更新后都应重新评估
   - 建立定期评估机制（如每周或每月）
   - 记录评估历史，追踪改进效果

3. **评估器选择**：
   - 根据应用场景选择适当的评估器
   - 检索系统重点关注Retrieval Precision和Recall
   - 生成系统重点关注Correctness、Groundedness和Relevance

4. **优化迭代**：
   - 根据评估结果定位问题
   - 逐步优化（先优化检索，再优化生成）
   - 使用A/B测试验证优化效果

### 8.4 RAGAS vs LangSmith对比

| 特性 | RAGAS | LangSmith |
|------|-------|-----------|
| **评估方式** | 无参考评估为主 | 支持有参考和无参考 |
| **平台** | 本地/云端 | 云端平台 |
| **可视化** | 基础可视化 | 强大的Web UI |
| **数据集管理** | 需要自己管理 | 内置数据集管理 |
| **实验追踪** | 需要自己实现 | 内置实验追踪 |
| **成本** | 主要是LLM调用成本 | LLM调用 + 平台费用 |
| **适用场景** | 快速评估、本地开发 | 生产环境、团队协作 |

**选择建议**：
- 开发阶段：可以使用RAGAS进行快速评估
- 生产环境：建议使用LangSmith进行持续监控
- 团队协作：LangSmith提供更好的协作功能

### 8.5 常见问题

**Q1: LangSmith评估需要多长时间？**
- A: 评估时间取决于数据集大小和LLM响应时间。LangSmith支持并发评估以提高效率。

**Q2: 如何降低评估成本？**
- A: 使用较便宜的LLM模型（如GPT-3.5）作为评估器；减少评估频率；使用采样评估。

**Q3: 评估结果不稳定怎么办？**
- A: 使用temperature=0确保一致性；多次评估取平均值；检查评估数据质量。

**Q4: 可以自定义评估指标吗？**
- A: 可以，LangSmith完全支持自定义评估器，你可以根据需求编写任何评估逻辑。

**Q5: 如何在生产环境中使用？**
- A: 将LangSmith追踪集成到RAG系统中，设置定期评估任务，监控关键指标。

---

## 总结

本教程介绍了如何使用LangSmith平台评估RAG系统的向量检索性能。通过Correctness、Groundedness、Relevance、Retrieval Precision和Retrieval Recall等评估器，我们可以全面了解RAG系统在检索和生成方面的表现。

**关键要点**：
- ✅ LangSmith提供完整的评估平台和可视化界面
- ✅ 支持多种评估器类型，可以灵活组合
- ✅ 内置数据集管理和实验追踪功能
- ✅ 适合生产环境的持续监控和优化

**下一步**：
- 在LangSmith平台创建账户并获取API密钥
- 将评估集成到你的RAG系统中
- 建立评估数据集和定期评估机制
- 根据评估结果持续优化系统性能
